<a href="https://colab.research.google.com/github/TranPhu1999/Wordle_solver/blob/main/wordle_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>